In [101]:
import random
import os
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoProcessor
from transformers import DataCollatorWithPadding
from transformers import AutoModel, TrainingArguments, Trainer
import pandas as pd
import json
# from transformers import RobertaTokenizer, RobertaModel

random.seed(42)
percent_to_train = .8

model_name = 'allenai/biomed_roberta_base'
dropout = .03



In [36]:
currentdir = os.getcwd() # ~/MeasEval/baselines
combopath_txt = os.path.join(currentdir, "../data/raw/combo/tsv/")

print("combopath_txt: ", combopath_txt)
combopath_annot = os.path.join(currentdir, "../data/raw/combo/tsv/")
print("combopath_annot :", combopath_annot)
interimpath = os.path.join(currentdir, "../data/interim/")
print("interimpath: ", interimpath)


combopath_txt:  /home/michelle/MeasEval/baselines/../data/raw/combo/tsv/
combopath_annot : /home/michelle/MeasEval/baselines/../data/raw/combo/tsv/
interimpath:  /home/michelle/MeasEval/baselines/../data/interim/


In [62]:
# if we want to replicate original splits
# train_interim = os.path.join(interimpath, "train/")
# print("train interim folder: ", train_interim)
# dev_interim = os.path.join(interimpath, "dev/")
# print("dev interim folder: ", dev_interim)
# test_interim = os.path.join(interimpath, "test/")
# print("test interim folder: ", test_interim)

train interim folder:  /home/michelle/MeasEval/baselines/../data/interim/train/
dev interim folder:  /home/michelle/MeasEval/baselines/../data/interim/dev/
test interim folder:  /home/michelle/MeasEval/baselines/../data/interim/test/


In [69]:
# ex = os.listdir(os.path.join(currentdir, "../data/raw/train/tsv/"))[0] 
# ex

'S0006322312001096-1136.tsv'

In [72]:
# sam's code
# train_tsvs = os.path.join(currentdir, "../data/raw/train/tsv/")
# df = pd.DataFrame()
# for fn in os.listdir(train_tsvs):
#     path = train_tsvs+fn
#     file = pd.read_csv(path,delimiter='\t',encoding='utf-8')
#     df = pd.concat([df, file],ignore_index=True)

In [ ]:

def csv_to_pandas(path):
    path = path
    csv_temp = pd.read_csv(path, header=0)
    csv_temp['other'] = csv_temp['other'].fillna("{}")

    # convert to dictionary
    def str_to_dict(x):
        if isinstance(x, str):
            dict_str = json.loads(x)  
        else:
            print("something not string")
            # x = x.astype(str)
            # dict_str = json.loads(x)
        return dict_str
        
    csv_temp['docId'] = csv_temp['docId'].astype('string')
    csv_temp['annotType'] = csv_temp['annotType'].astype('string')
    csv_temp['text'] = csv_temp['text'].astype('string')
    csv_temp['annotId'] = csv_temp['annotId'].astype('string')
    csv_temp['other'] = csv_temp['other'].apply(str_to_dict, convert_dtype=True)

    return csv_temp
    
csv_to_pandas(os.path.join(currentdir, "../data/interim/train_annot.csv"))

In [ ]:
def generate_pd_dfs(train_path, dev_path, test_path):
    train_path = train_path
    dev_path = dev_path
    test_path = test_path

    train_df = csv_to_pandas(train_path)
    dev_df = csv_to_pandas(dev_path)
    test_df = csv_to_pandas(test_path)

    return train_df, dev_df, test_df

In [161]:
path = os.path.join(currentdir, "../data/interim/train_annot.csv")


In [162]:
csv_test.head()

,Unnamed: 0,docId,annotSet,annotType,startOffset,endOffset,annotId,text,other
0,0,S0012821X12004384-1610,1,Quantity,90,98,T1-1,2617.4 m,"{""unit"": ""m""}"
1,1,S0012821X12004384-1610,1,MeasuredEntity,4,14,T3-1,brief peak,"{""HasQuantity"": ""T1-1""}"
2,2,S0012821X12004384-1610,1,Qualifier,15,30,T4-1,in Apectodinium,"{""Qualifies"": ""T3-1""}"
3,3,S0012821X12004384-1610,2,Quantity,669,688,T1-2,2619.6 and 2614.7 m,"{""mods"": [""IsList""], ""unit"": ""m""}"
4,4,S0012821X12004384-1610,2,MeasuredEntity,638,649,T3-2,other peaks,"{""HasQuantity"": ""T1-2""}"


In [163]:
# check for NaNs
csv_test[csv_test['other'].apply(lambda x: isinstance(x, float))]

,Unnamed: 0,docId,annotSet,annotType,startOffset,endOffset,annotId,text,other
265,376,S0032063313003218-5269,4,Quantity,352,366,T1-4,on September 8,NaN
272,383,S0032063313003218-5269,6,Quantity,386,399,T1-6,September 11),NaN
297,416,S0032063313003218-7156,5,Quantity,389,407,T1-5,three times larger,NaN
470,657,S0167880913001229-1323,4,Quantity,549,553,T1-4,0.07,NaN
505,715,S0257897213007573-899,2,Quantity,712,716,T1-2,1.32,NaN
...,...,...,...,...,...,...,...,...,...
3413,4744,S2213671113000738-738,2,Quantity,544,552,T1-2,6 months,NaN
3416,4747,S2213671113000738-738,3,Quantity,867,871,T1-3,1/10,NaN
3425,4756,S2213671113000738-738,6,Quantity,1903,1908,T1-6,1/100,NaN
3440,4782,S2213671113000921-994,4,Quantity,685,688,T1-4,one,NaN


In [164]:
# replace NaN values with correct dtypes
csv_test['other'] = csv_test['other'].fillna("{}")
csv_test[csv_test['other'].apply(lambda x: isinstance(x, float))]

,Unnamed: 0,docId,annotSet,annotType,startOffset,endOffset,annotId,text,other


In [165]:
# convert to dtypes
def str_to_dict(x):
    if isinstance(x, str):
        dict_str = json.loads(x)
        
    else:
        x = x.astype(str)
        dict_str = json.loads(x)

    return dict_str
        
csv_test['docId'] = csv_test['docId'].astype('string')
csv_test['annotType'] = csv_test['annotType'].astype('string')
csv_test['text'] = csv_test['text'].astype('string')
csv_test['annotId'] = csv_test['annotId'].astype('string')
csv_test['other'] = csv_test['other'].apply(str_to_dict, convert_dtype=True)


In [166]:
# test replacement type
a = csv_test['other'].iloc[265]
print(type(a))

<class 'dict'>


In [167]:
print(csv_test.dtypes)

    


Unnamed: 0      int64
docId          string
annotSet        int64
annotType      string
startOffset     int64
endOffset       int64
annotId        string
text           string
other          object
dtype: object


In [70]:

with 
    path = os.listdir(os.path.join(currentdir, "../data/raw/train/tsv/"))[0]
    
    
    print(len(csv_test))


    df = pd.DataFrame()
    for fn in os.listdir(train_tsvs):
        path = train_tsvs+fn
        file = pd.read_csv(path,delimiter='\t',encoding='utf-8')
        df = pd.concat([df, file],ignore_index=True)


encoding='utf-8'
# results = pd.read_csv(os.listdir()

AttributeError: __enter__

In [ ]:
# raw/train/text
# raw/train/tsv

for fn in os.listdir(combopath_annot):
    with open()
    pd.read_csv('c:/~/trainSetRel3.txt', sep='\t', header=0)


In [57]:
print(combopath_annot)
len(os.listdir(combopath_annot))

/home/michelle/MeasEval/baselines/../data/raw/combo/tsv/


428

In [52]:
len(os.listdir(os.path.join(currentdir, "../../train/tsv/")))

233

In [59]:
len(os.listdir(os.path.join(currentdir, "../data/raw/train/text/")))

248

In [54]:
len(os.listdir(os.path.join(currentdir, "../data/raw/trial/tsv/")))

65

In [60]:
len(os.listdir(os.path.join(currentdir, "../data/raw/eval/tsv/")))

130

In [49]:
for fn in os.listdir(combopath_annot):
    print(fn)

S0006322312001096-1136.tsv
S0006322312001096-1177.tsv
S0006322312001096-1190.tsv
S0006322312001096-1194.tsv
S0006322312001096-1197.tsv
S0006322312001096-1202.tsv
S0006322312001096-1205.tsv
S0006322312001096-1221.tsv
S0006322312001096-1230.tsv
S0006322312001096-1248.tsv
S0006322312001096-1253.tsv
S0006322312001096-1260.tsv
S0006322312001096-1271.tsv
S0006322312001096-1275.tsv
S0006322312001096-1278.tsv
S0006322312001096-626.tsv
S0012821X12004384-1148.tsv
S0012821X12004384-1178.tsv
S0012821X12004384-1221.tsv
S0012821X12004384-1232.tsv
S0012821X12004384-1249.tsv
S0012821X12004384-1265.tsv
S0012821X12004384-1284.tsv
S0012821X12004384-1302.tsv
S0012821X12004384-1405.tsv
S0012821X12004384-1415.tsv
S0012821X12004384-1594.tsv
S0012821X12004384-1599.tsv
S0012821X12004384-1610.tsv
S0012821X12004384-1640.tsv
S0012821X12004384-952.tsv
S0012821X12004384-990.tsv
S0012821X13002185-1061.tsv
S0012821X13002185-1200.tsv
S0012821X13002185-1217.tsv
S0012821X13002185-1231.tsv
S0012821X13002185-835.tsv
S0012

In [39]:

if reshuffle_docs == True:
    combo_txt = {}
    for fn in os.listdir(combopath_txt):
        docIds.append(fn[:-4])
        path = combopath_txt+fn
        with open(path) as textfile:
                text = textfile.read()
                #[:-4] strips off the .txt to get the id
                combo_txt[fn[:-4]] = text

if reshuffle_docs == True:
    combo_annot = pd.DataFrame()
    for fn in os.listdir(combopath_annot):
        path = combopath_annot+fn
        file = pd.read_csv(path,delimiter='\t',encoding='utf-8')
        combo_annot = pd.concat([combo_annot, file],ignore_index=True)

    random.shuffle(docIds)

    n_doc = len(docIds)
    split_train = int(np.round(n_doc * percent_to_train))
    split_dev = split_train + int(np.round(n_doc * percent_to_dev))

    docs_train = docIds[:split_train]
    docs_dev = docIds[split_train:split_dev]
    docs_test = docIds[split_dev:]

    train_annot = combo_annot.loc[combo_annot['docId'].isin(docs_train)]
    dev_annot = combo_annot.loc[combo_annot['docId'].isin(docs_dev)]
    test_annot = combo_annot.loc[combo_annot['docId'].isin(docs_test)]

    # save data
    train_annot.to_csv(interimpath+'train_annot.csv')
    dev_annot.to_csv(interimpath+'dev_annot.csv')
    test_annot.to_csv(interimpath+'test_annot.csv')

    train_txt = {d: combo_txt[d] for d in docs_train}
    dev_txt = {d: combo_txt[d] for d in docs_dev}
    test_txt = {d: combo_txt[d] for d in docs_test}
    
    with open(interimpath+'train_txt.json','w') as f:
        json.dump(train_txt, f)
    with open(interimpath+'dev_txt.json','w') as f:
        json.dump(dev_txt, f)
    with open(interimpath+'test_txt.json','w') as f:
        json.dump(test_txt, f)

else:
    train_annot = pd.read_csv(interimpath+'train_annot.csv')
    dev_annot = (interimpath+'dev_annot.csv')
    test_annot = (interimpath+'test_annot.csv')

    with open(interimpath+'train_txt.json','r') as f:
        train_txt = json.load(f)
    with open(interimpath+'dev_txt.json','r') as f:
        dev_txt = json.load(f)
    with open(interimpath+'test_txt.json','r') as f:
        test_txt = json.load(f)